<a href="https://colab.research.google.com/github/RyanLe2003/Favorite-Artist-Song-Lyric-Generator/blob/main/Song_Lyric_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("This is a song lyrics generator!")
print("--------------------------------")
print("You will enter the name of your favorite artist and the generator will produce a song that sounds like it could be one of the artist's'")
print("The process may take some time and make sure your spelling is correct")
print("")
artist = input("Enter the name of your favorite artist: ").lower().replace(" ", "-")

In [6]:
import requests
import time
from bs4 import BeautifulSoup

#scrape favorite artist site for all hyperlinks
url = f"https://www.songlyrics.com/{artist}-lyrics/" 
page = requests.get(url)
homepage = BeautifulSoup(page.text, "html.parser")

#find individual hyperlinks and add them to the list
entries = homepage.find_all("a", attrs={"itemprop": "url"})

all_links = []
for entry in entries:
  all_links.append(entry.attrs["href"])

#append the lyrics from each hyperlink(unique song) to a list
lyrics = []
for link in all_links:
    response = requests.get(link)
    individual_song_soup = BeautifulSoup(response.text, "html.parser")
    individual_lyrics = individual_song_soup.find("p", attrs={"id": "songLyricsDiv", "class": "songLyricsV14 iComment-text"})
    individual_lyrics= str(individual_lyrics).replace('<p class="songLyricsV14 iComment-text" id="songLyricsDiv">', "") #clean data
    individual_lyrics = str(individual_lyrics).replace('</p>', '') #clean data
    lyrics.append(str(individual_lyrics))
    time.sleep(0.1) #add to prevent getting banned from site

In [7]:
import pickle
pickle.dump(lyrics, open("lyrics.pkl", "wb"))

In [8]:
def train_markov_chain(lyrics):
    """
    Args:
      - lyrics: a list of strings, where each string represents
                the lyrics of one song by an artist.
    
    Returns:
      A dict that maps a tuple of 2 words ("bigram") to a list of
      words that follow that bigram, representing the Markov
      chain trained on the lyrics.
    """
    chain = {(None, "<START>"): []}
    
    for lyric in lyrics:
        
        line = lyric.replace("\n", " <N> ") 
        line = line.split()
        
        previous_word1 = None
        previous_word2 = "<START>"
        for word in line:
          bigram = (previous_word1, previous_word2)
          if bigram not in chain:
            chain[bigram] = []
          chain[bigram].append(word)
          previous_word1 = previous_word2
          previous_word2 = word

        bigram = (previous_word1, previous_word2)
        if bigram not in chain:
          chain[bigram] = []
        chain[bigram].append("<END>")
          
    return chain

In [ ]:
# Load the pickled lyrics object
import pickle
lyrics = pickle.load(open("lyrics.pkl", "rb"))

# Call the function you wrote above.
chain = train_markov_chain(lyrics)

# What words tend to start a song
print(chain[(None, "<START>")])

In [10]:
import random

def generate_new_lyrics(chain):
    """
    Args:
      - chain: a dict representing the Markov chain,
               such as one generated by generate_new_lyrics()
    
    Returns:
      A string representing the randomly generated song.
    """
    
    # a list for storing the generated words
    words = []
    
    # generate the first word
    words.append(random.choice(chain[(None, "<START>")]))
    
    #generate other words
    first = "<START>"
    second = words[-1]
    
    while "<END>" not in words:
        next_word = random.choice(chain[(first, second)])
        words.append(next_word)
        first = second
        second = next_word
    
    # join the words together into a string with line breaks
    lyrics = " ".join(words[:-1])
    return "\n".join(lyrics.split("<N>"))

In [ ]:
print("run this cell as many times you want to generate new songs")
print("")
print(generate_new_lyrics(chain))